1. Install Required Libraries in Google Colab
First, need to install the necessary libraries. This can be done directly within Google Colab using !pip install.

In [ ]:
# Install necessary libraries
!pip install flask tensorflow scikit-learn py-solc-x requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2


2. Setup Blockchain (Using Ethereum Smart Contracts). For this example, we will use web3.py to interact with the Ethereum blockchain.

In [ ]:
# Install web3.py to interact with Ethereum
!pip install web3

from web3 import Web3

# Connect to Ethereum node (Local or Infura)
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/c638172c957e49259abd5bd1325d3981'))

# Check if connection is successful
# In newer versions of web3.py, isConnected is replaced by is_connected
print(f"Connected to Ethereum: {w3.is_connected()}")


Connected to Ethereum: True


3. Deploy a Simple Ethereum Smart Contract
Next, we’ll write a simple smart contract in Solidity and deploy it on the Ethereum blockchain. You can deploy to a test network like Ropsten or Rinkeby for testing purposes.

In [ ]:
// Solidity Smart Contract Example

pragma solidity ^0.8.0;

contract ComplianceCheck {
    address public owner;

    mapping(address => bool) public isCompliant;

    event ComplianceStatus(address indexed user, bool status);

    constructor() {
        owner = msg.sender;
    }

    function setCompliance(address user, bool status) public {
        require(msg.sender == owner, "Only owner can set compliance status");
        isCompliant[user] = status;
        emit ComplianceStatus(user, status);
    }

    function checkCompliance(address user) public view returns (bool) {
        return isCompliant[user];
    }
}


4. Integrate AI/ML for Risk Assessment (Sample Model).
We'll use a simple ML model to detect anomalies in transaction data. We’ll use scikit-learn to build a basic model that classifies transactions as "risky" or "safe."

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Sample transaction data
data = {
    'transaction_id': [1, 2, 3, 4, 5],
    'amount': [1000, 2500, 800, 7000, 500],
    'location': ['US', 'US', 'US', 'UK', 'IN'],
    'is_risky': [0, 1, 0, 1, 0]  # 1 = risky, 0 = safe
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Feature engineering (basic encoding for demo)
df['location'] = df['location'].map({'US': 0, 'UK': 1, 'IN': 2})

# Define features and target
X = df[['amount', 'location']]
y = df['is_risky']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a RandomForest model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


Accuracy: 0.5


5. Create a Simple API with Flask
Now, we will create a simple Flask API that will serve the smart contract and AI/ML model together. The API will:

Take transaction data as input.
Use the AI model to assess risk.
Check compliance via the smart contract.

In [ ]:
# Install Flask if it's not installed
!pip install Flask

from flask import Flask, request, jsonify
import random

app = Flask(__name__)

# Mock AI risk assessment function (using the trained model above)
def predict_risk(amount, location):
    prediction = model.predict([[amount, location]])  # RandomForest from above
    return "risky" if prediction == 1 else "safe"

# Mock smart contract function
def check_compliance(user_address):
    # Normally you would interact with the Ethereum blockchain here
    return random.choice([True, False])  # Random compliance status

@app.route('/assess', methods=['POST'])
def assess_transaction():
    data = request.get_json()

    # Extract data
    amount = data.get('amount')
    location = data.get('location')
    user_address = data.get('user_address')

    # Risk assessment
    risk = predict_risk(amount, location)

    # Compliance check
    compliance = check_compliance(user_address)

    # Return results
    return jsonify({
        'risk': risk,
        'compliance': compliance
    })

# Run the Flask app
if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


6. Test API Locally in Google Colab
Since Colab does not support hosting Flask directly, you will need to use ngrok for tunneling. Here's how you can do it.

In [ ]:
# Install ngrok
!pip install pyngrok

from pyngrok import ngrok

# Start Flask app in a background thread
port = 5000
public_url = ngrok.connect(port)
print(f"API available at {public_url}")


7. Example Request to API
You can now send requests to the API. Here's how you would call the endpoint from Google Colab or any HTTP client like Postman:

In [ ]:
import requests

# Sample data for testing the API
data = {
    "amount": 1000,
    "location": 0,  # US
    "user_address": "0x123abc..."
}

response = requests.post('http://localhost:5000/assess', json=data)
print(response.json())


Conclusion
This Google Colab notebook demonstrates how to integrate AI/ML models, blockchain smart contracts, and Flask to create an API for compliance and risk management. You can extend this by deploying the Flask app on a cloud server, integrating with a real blockchain, and building more sophisticated AI/ML models.